In [2]:
import numpy as np
import pandas as pd

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler,LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline, make_pipeline

In [6]:
df = pd.read_csv('covid_toy.csv')

In [40]:
df.drop(columns=['city'],inplace=True)

In [41]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']),
                                                    df['has_covid'],
                                                    test_size=0.3,
                                                    random_state=0)
x_train.shape

(70, 4)

In [51]:
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)

In [42]:
tnf1 = ColumnTransformer([
    ('si_fever', SimpleImputer(),[2])
],remainder='passthrough')

In [43]:
tnf2 = ColumnTransformer(transformers=[
    ('ohe_gender', OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1]),
    ('ode_cough', OrdinalEncoder(categories=[['Mild','Strong']]),[3])
],remainder='passthrough')

[('ohe_gender_city', OneHotEncoder(handle_unknown='ignore', sparse_output=False), [1, 4]), ('ode_cough', OrdinalEncoder(categories=[['Mild', 'Strong']]), [3])]


In [22]:
tnf3 = ColumnTransformer([
    ('scaling', MinMaxScaler(),[0,2])
])

In [34]:
tnf5 = SelectKBest(score_func=chi2, k=5)

In [44]:
tnf4 = DecisionTreeClassifier()

In [47]:
pipe = make_pipeline(tnf1,tnf2,tnf3,tnf4)

In [48]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('si_fever', SimpleImputer(),
                                                  [2])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_gender',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1]),
                                                 ('ode_cough',
                                                  OrdinalEncoder(categories=[['Mild',
                                                                              'Strong']]),
                                                  [3])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                                  [0, 2])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [49]:
y_pred = pipe.predict(x_test)

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.36666666666666664